In [9]:
import json
from datetime import datetime
import math as m
import pandas as pd
import pymysql as pm
from sqlalchemy import create_engine
from util.slack import Slack
import numpy as np

In [10]:
config_path = './config/config.json'
sql_path = './sql/'

# load config
config = json.load(open(config_path))
    
# Slack client
slack = Slack(config['slack']['channel'], config['slack']['token'])

# load mySQL instance (for select)
db = pm.connect(
    user=config['mysqlCardSlave']['user'], 
    passwd=config['mysqlCardSlave']['password'], 
    host=config['mysqlCardSlave']['host'], 
    db=config['mysqlCardSlave']['database'], 
    port=config['mysqlCardSlave']['port'],
    cursorclass=pm.cursors.DictCursor)
cursor = db.cursor()

f = open(str(sql_path)+"special_limit_request.sql", 'r')

sql = ''
while True:
    line = f.readline()
    if not line: break
    a = str(line)
    sql = sql + a
f.close()

cursor.execute(sql)
result = cursor.fetchall()
db.close()

df = pd.DataFrame(result)

print(df)

for idxCorp in df['idxCorp']:
    mask=df['idxCorp']==idxCorp
    slack_msg=( '▶︎ 법인개요 ◀︎'+'\n'+
                '법인명: {}'+'\n'+
                '희망한도: {}'+'\n'+
                '심사요청일: {}'+'\n'+
                '마감예정일: {}'+'\n'
                ).format(
                    df.loc[mask]['법인명'].item(),
                    df.loc[mask]['희망한도'].item(),
                    df.loc[mask]['심사요청일'].item(),
                    df.loc[mask]['마감예정일'].item())
    slack.send(title='특별심사_Backlog', message=slack_msg)



   idxCorp         법인명         희망한도       심사요청일       마감예정일
0     8155    주식회사 버틀러   10,000,000  2022-09-16  2022-09-21
1     8164  주식회사 탑픽이엔엠   20,000,000  2022-09-15  2022-09-20
2     8191  엠피엔컴퍼니주식회사  100,000,000  2022-09-19  2022-09-24


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/slack_sdk/web/internal_utils.py:264: UserWarning: The top-level `text` argument is missing in the request payload for a chat.postMessage call - It's a best practice to always provide a `text` argument when posting a message. The `text` argument is used in places where content cannot be rendered such as: system push notifications, assistive technology such as screen readers, etc.
  warnings.warn(message, UserWarning)
